In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cuml

In [ ]:
train_path = '../input/analytics-vidhya-jobathon1-2022/train_wn75k28.csv'
test_path = '../input/analytics-vidhya-jobathon1-2022/test_Wf7sxXF.csv'
sub_path = '../input/analytics-vidhya-jobathon1-2022/sample_submission_2zvVjBu.csv'

In [ ]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_sub = pd.read_csv(sub_path)

In [ ]:
df_train.info()

In [ ]:
df_train['user_activity_var_9'].value_counts()

In [ ]:
df_test['user_activity_var_9'].value_counts()

In [ ]:
df_train['indicator_signup_date'] = df_train['signup_date'].isnull()
df_train['indicator_products_purchased'] = df_train['products_purchased'].isnull()
df_test['indicator_signup_date'] = df_test['signup_date'].isnull()
df_test['indicator_products_purchased'] = df_test['products_purchased'].isnull()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_train['indicator_signup_date'] = le.fit_transform(df_train['indicator_signup_date'].astype(str))  
df_train['indicator_products_purchased'] = le.fit_transform(df_train['indicator_products_purchased'].astype(str))  
df_test['indicator_signup_date'] = le.fit_transform(df_test['indicator_signup_date'].astype(str))  
df_test['indicator_products_purchased'] = le.fit_transform(df_test['indicator_products_purchased'].astype(str)) 

In [ ]:
df_train['created_at'] = pd.to_datetime(df_train['created_at'])
df_train['signup_date'] = pd.to_datetime(df_train['signup_date'])

In [ ]:
df_test['created_at'] = pd.to_datetime(df_test['created_at'])
df_test['signup_date'] = pd.to_datetime(df_test['signup_date'])

In [ ]:
df_train['products_purchased'] = df_train['products_purchased'].fillna(0)
df_test['products_purchased'] = df_test['products_purchased'].fillna(0)

In [ ]:
combo = pd.concat([df_train.drop('buy',axis=1),df_test], ignore_index=False)
combo.drop(['id','created_at','signup_date'],axis=1,inplace=True)
combo

In [ ]:
from cuml.cluster import KMeans
Sum_of_squared_distances = []
K = range(1,10)
for num_clusters in K :
 kmeans = KMeans(n_clusters=num_clusters)
 kmeans.fit(combo)
 Sum_of_squared_distances.append(kmeans.inertia_)
plt.plot(K,Sum_of_squared_distances)
plt.xlabel('Values of K') 
plt.ylabel('Sum of squared distances/Inertia') 
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
k_means = KMeans(n_clusters=2)
k_means.fit(combo)

labels = k_means.predict(combo)
combo['label'] = labels

In [ ]:
combo['label'].value_counts()

In [ ]:
df_train['label'] = combo['label'].head(df_train.shape[0])
df_test['label'] = combo['label'].tail(df_test.shape[0])

In [ ]:
df_train['camapign_var'] = df_train['campaign_var_1'] + df_train['campaign_var_2']
df_test['camapign_var'] = df_test['campaign_var_1'] + df_test['campaign_var_2']

In [ ]:
dumm_feat = ['id','created_at','signup_date','buy','camapign_var','campaign_var_1','campaign_var_2'] # ,'products_purchased'
CAT_FEATURE = [feat for feat in df_train.columns if feat not in dumm_feat]
CAT_FEATURE

In [ ]:
for feat in CAT_FEATURE:
    df_train[feat] = df_train[feat].astype('int64')
    df_test[feat] = df_test[feat].astype('int64')

In [ ]:
train = df_train.copy()
x = train.drop(['id','created_at','signup_date','buy'],axis=1) # 'id',
y = df_train['buy']
test = df_test.drop(['id','created_at','signup_date'],axis=1) # 'id',

In [ ]:
from sklearn.model_selection import train_test_split
x_train ,x_test , y_train ,y_test = train_test_split(x,y,stratify=y,test_size=0.2,random_state=101)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score
from sklearn.metrics import classification_report

xgbc = XGBClassifier(n_estimators = 39)

xgbc.fit(x_train,y_train,eval_set=[[x_train,y_train],[x_test,y_test]])

y_test_preds_proba_xgbc = xgbc.predict_proba(x_test)
y_test_preds_xgbc = xgbc.predict(x_test)
print()     
print('F1 Score   '+str(f1_score(y_test, y_test_preds_xgbc)))
print()
print(classification_report(y_test, y_test_preds_xgbc))

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score,f1_score

cbc = CatBoostClassifier(eval_metric='TotalF1',cat_features=CAT_FEATURE)

cbc.fit(x_train,y_train,eval_set=(x_test,y_test))

y_test_preds_proba_cbc = cbc.predict_proba(x_test)
y_test_preds_cbc = cbc.predict(x_test)
print()
print('F1 Score   '+str(f1_score(y_test, y_test_preds_cbc)))
print()
print(classification_report(y_test, y_test_preds_cbc))

In [ ]:
import lightgbm as lgb
lgc = lgb.LGBMClassifier()

lgc.fit(x_train,y_train,eval_set=[[x_train,y_train],[x_test,y_test]])

y_test_preds_proba_lgc = lgc.predict_proba(x_test)
y_test_preds_lgc = lgc.predict(x_test)

print('F1 Score   '+str(f1_score(y_test, y_test_preds_lgc)))
print()
print(classification_report(y_test, y_test_preds_lgc))

In [ ]:
from sklearn.ensemble import StackingClassifier
stk = StackingClassifier([('cbc',CatBoostClassifier(eval_metric='TotalF1',cat_features=CAT_FEATURE)),
                          ('lgc',lgb.LGBMClassifier()),
                          ('xgb',XGBClassifier())])
stk.fit(x_train,y_train)
preds_valid = stk.predict(x_test)

print(classification_report(y_test,preds_valid))

In [ ]:
preds_test = stk.predict(test)
df_test['buy'] = 0
df_test['buy'] = preds_test

In [ ]:
# preds_test = cbc.predict(test)
df_sub['buy']= df_test['buy']

df_sub.to_csv('submission.csv',index=False)

In [ ]:
import seaborn as sns
sns.heatmap(df_train.corr())